## Где мы?

Добро пожаловать в **Jupyter Notebook** - аналог Python Interactive, позволяющий запускать код блоками.

Здесь можно писать всякие формулы (LaTeX):

$\sum a_x$

а также есть поддержка картинок/гифок:

<img src = "img/numpy1.jpg" width="400">

## Что такое NumPy и почему он крутой

Основная причина, по которой принято говорить, что классический Python медленный - это **динамическая типизация.**

In [2]:
a = 5 #присвоим переменной целочисленный тип
a = "some string" #а затем сразу переопределим её строковым типом

Для сравнения, в C++ мы заранее указываем, какой тип данных (и только его) содержит переменная.

"Магия" такого поведения Python, как ни странно, скрывается под капотом, где каждая переменная представляет собой структуру языка C. В памяти массив представляется как-то так:

<img src = "img/numpy2.jpg" width="400">

Формально это означает, что элементы массива содержатся где попало, что приводит к нерациональному использованию памяти.

Использование NumPy делает управление памятью более рациональным, однако лишает нас динамической типизации. Подробнее рассмотрим это чуть позже. А сейчас оценим время, которое позволяет экономить использование NumPy

In [3]:
import numpy as np
import time 

python_array = [i for i in range(100000000)]
numpy_array = np.array(python_array)

start = time.time()
for i in python_array:
    i += 1
print("Python работал {} секунд".format(time.time() - start))

start = time.time()
numpy_array += 1
print("Numpy Работал {} секунд".format(time.time() - start))

Python работал 6.901902914047241 секунд
Numpy Работал 0.054979801177978516 секунд


Тут нужно сказать пару слов про технологию векторизации, которую использует NumPy.

**SIMD (Single Instruction, Multiple Data)** позволяет процессору выполнять одну и ту же операцию сразу над несколькими элементами данных. В классической же модели обработки каждый эл-т обрабатывается отдельно (т.е. одно значение за одну единицу времени).

NumPy **не является прямым интерфейсом SIMD,** однако использует такие низкоуровневые библиотеки, как BLAS (Basic Linear Algebra Subprograms) и LAPACK (Linear Algebra PACKage), которые оптимизированы с помощью SIMD.

## Это база

Рассмотрим самые базовые команды, которые предлагает нам данная библиотека.

In [4]:
np.array([1, 2, 3, 4, 5]) #создание массива NumPy

array([1, 2, 3, 4, 5])

In [5]:
print(np.zeros(5)); print(np.ones(5))

[0. 0. 0. 0. 0.]
[1. 1. 1. 1. 1.]


In [6]:
print(np.arange(0, 10, 2)); print(np.linspace(0, 5, 10))

[0 2 4 6 8]
[0.         0.55555556 1.11111111 1.66666667 2.22222222 2.77777778
 3.33333333 3.88888889 4.44444444 5.        ]


(arange создаёт элементы от 0 до 10 с шагом 2, а linspace 10 элементов от 0 до 5) 

In [8]:
np.identity(5) #единичная матрица

array([[1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 1., 0., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 1.]])

In [10]:
a = np.array([1, 2, 3, 4, 5])
print(a.shape, a.dtype)

(5,) int64


In [11]:
a = np.array([2, 3, 4, 1, 5])
np.sort(a) #По умолчанию - QuickSort

array([1, 2, 3, 4, 5])

In [14]:
a = np.array([[1, 2], [3, 4]], dtype = float) #можно явно указать тип данных
b = a.flatten()
b

array([1., 2., 3., 4.])

Ну и не забудем упомянуть про всякие математические приколы:

In [16]:
print(np.e, np.pi, np.log(np.e))

2.718281828459045 3.141592653589793 1.0


А ещё пару строк про векторные операции в NumPy:

In [17]:
print(np.array([1, 2, 3, 4]) + 1)

[2 3 4 5]


In [18]:
a = np.array([1, 2, 3])
b = np.array([4, 5, 6])
a * b

array([ 4, 10, 18])

## UFUNC и аггрегирование

В NumPy у каждой операции есть свой собственный аналог, который называется **ufunc (универсальные функции).** Они оптимизированы и имеют свои вспомогательные функции, но обо всё по порядку.

In [20]:
a + b        # абсолютно
np.add(a, b) # одинаковые записи

array([5, 7, 9])

In [21]:
np.add.accumulate(a) #возвращает все шаги сумирования слева направо (a = [1, 2, 3])

array([1, 3, 6])

In [22]:
np.add.reduce(a) #вернёт сумму массива

6

и так далее и так далее. С полным списком вспомогательных функции можно ознакомиться в документации.

Часто при работе с данными пригождается собрать какую-то первичную статистику, т.е. выявление максимума, минимума по определенным признакам. Конечно же, для этого лучше использовать встроенные в NumPy функции:

In [24]:
a = np.arange(10)
min_val = np.min(a)
max_val = np.max(a)
mean_val = np.mean(a)
sum_val = np.sum(a)
print("Min:{}\nMax:{}\nMean:{}\nSum:{}".format(min_val, max_val, mean_val, sum_val))

Min:0
Max:9
Mean:4.5
Sum:45


In [25]:
b = np.arange(9).reshape(3, 3) #переопределим размеры массива (теперь это матрица 3х3)
min_val1 = np.min(b, axis = 0)
min_val1

array([0, 1, 2])

Здесь стоит немного остановиться. Для функции можно указывать ось, по которой мы хотим её применить, однако работает она не так однозначно.
Для понимания добавлю, что **axis = 0 - строки, axis = 1 - столбцы в случае двумерных массивов.**

Ключевое слово axis показывает, какая ось будет **СХЛОПНУТА,** а не по какой будет применяться функция. Опять же, рассмотрим массив b:

In [26]:
b

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

Указывая для np.min axis = 0 мы говорим, что ось строк будет схлопнута, и функция будет применяться по столбцам. Так и выходит - min_val1 является массивом, который содержит минимальные элементы каждого столбца исходного массива b.

## Broadcasting rules

**Правила транслирования** помогают складывать массивы, матрицы и возможные их комбинации между собой. Эти правила состоят из 3-х пунктов, выполнение (или не выполнение) которых определяют порядок сложения (бинарной операции) массивов.

In [27]:
a = np.array([1, 2])
b = np.array([[1, 2],
              [3, 4],
              [5, 6]])

print(a.shape, b.shape)

(2,) (3, 2)


Обратите внимания на "лишнюю" запятую у размерности a.

Итак, правила транслирования:

1. Если количество измерений (элементов в shape) не совпадает, то отстающему массиву дописывается ведущая единица **слева.** Для наших массивов - (2,) $\rightarrow$ (1, 2)
2. Если в каком-то конкретном измерении один из массивов отстаёт от второго, и при этом "длина" этого измерения **равна 1,** тогда она "растягивается" до нужного значения. Для наших массивов - (1, 2) $\rightarrow$ (3, 2).

На втором шаге уже становится понятно, что мы можем сложить эти самые массивы, однако есть ещё третье правило:

3. В противных случаях - ошибка.

In [28]:
a + b

array([[2, 4],
       [4, 6],
       [6, 8]])

Если говорить более простым языком, массив a поддаётся правилам транслирования, а значит растягивается до размеров b, при этом дублируя необходимые элементы. Таким образом, **к каждой строке массива b прибавилась строка a.**

## "Причудливая" индексация

(Я правда пытался понять, почему она называется именно так, я не нашёл :( )

Тем не менее, в классическом Python мы привыкли индексировать как-то так:

In [32]:
array = [1, 2, 3, 4]
print(a[1])
print(a[-1])
print(a[:2])

2
2
[1 2]


NumPy же предлагает слегка продвинутую версию: что если мы в качестве индекса будем передавать не число, а сразу массив?

In [33]:
a = np.arange(10)
index_ar = [1, 5, -1]
print(a[index_ar])

[1 5 9]


Мы можем комбинировать все виды индексирования:

In [35]:
b = np.arange(9).reshape(3, 3)
b

array([[0, 1, 2],
       [3, 4, 5],
       [6, 7, 8]])

In [36]:
index_ar = [0, 2]
print(b[1][index_ar])

[3 5]


## Операция маскирования

Рассмотрим, как NumPy синергирует с логическими операциями:

In [37]:
a = np.array([0, 3, 4, 2, 10, 9, 2, 6])
print(a > 3)

[False False  True False  True  True False  True]


Логично, что False - неудовлетворяющие значения, а True - угадайте какие. Разве что добавлю, что в NumPy False интерпретируется как 0, а True - как 1.

Думаю многие уже догадались, как это можно применять на практике:

In [38]:
a[a > 3]

array([ 4, 10,  9,  6])

Т.е. возвращаются элементы, удовлетворяющие условию. В квадратных скобках может быть выражение любой сложности.

Возвращаясь к теме выше, повторюсь, что мы можем комбинировать **любые методы индексирования.**